In [1]:
pip install --user opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [23]:
import cv2
import numpy as np
import csv

# Function to convert a rotation matrix to Euler angles
def rotationMatrixToEulerAngles(R):
    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.degrees(x), np.degrees(y), np.degrees(z)



In [ ]:
# Load the baseline CSV data
def load_baseline_csv(csv_file):
    baseline_data = []
    with open(csv_file, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row
        for row in csv_reader:
            frame_id = int(row[0])
            qr_id = int(row[1])
            qr_2d = eval(row[2])  # Convert string to list
            qr_3d = eval(row[3])  # Convert string to list
            baseline_data.append((frame_id, qr_id, qr_2d, qr_3d))
    return baseline_data


In [ ]:
# Calculate movement commands based on current and target poses
def calculate_movement_commands(current_pose, target_pose):
    # Placeholder logic for calculating movement commands
    commands = []
    # Calculate differences in yaw, pitch, roll
    yaw_diff = target_pose[1] - current_pose[0]
    pitch_diff = target_pose[2] - current_pose[1]
    roll_diff = target_pose[3] - current_pose[2]
    # Generate movement commands based on differences
    if yaw_diff > 5:
        commands.append("turn-right")
    elif yaw_diff < -5:
        commands.append("turn-left")
    if pitch_diff > 5:
        commands.append("down")
    elif pitch_diff < -5:
        commands.append("up")
    if roll_diff > 5:
        commands.append("right")
    elif roll_diff < -5:
        commands.append("left")
    return commands




In [ ]:
# Main function for processing live video and generating commands
def process_live_video(baseline_data):
    # Initialize video capture
    cap = cv2.VideoCapture(0)  # Use 0 for default webcam
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect ArUco markers in the frame
        corners, ids, rejected = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
        
        if ids is not None:
            for i in range(len(ids)):
                id = ids[i][0]
                # Find matching baseline data for the current frame
                for baseline_entry in baseline_data:
                    if baseline_entry[1] == id:  # Matching marker ID
                        baseline_pose = baseline_entry[3]  # [distance, yaw, pitch, roll]
                        
                        # Estimate pose of the detected marker
                        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners[i], marker_length, camera_matrix, dist_coeffs)
                        rvec = rvecs[0]
                        R, _ = cv2.Rodrigues(rvec)
                        current_pose = rotationMatrixToEulerAngles(R)  # [yaw, pitch, roll]
                        
                        # Calculate movement commands
                        commands = calculate_movement_commands(current_pose, baseline_pose)
                        
                        # Print commands (replace with actual drone control)
                        if commands:
                            print("Movement Commands:", commands)
        
        # Display the frame
        cv2.imshow('Live Video', frame)
        
        # Check for user input to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == '__main__':
    # Load the ArUco dictionary
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
    # Create detector parameters
    parameters = cv2.aruco.DetectorParameters()

    # Load camera calibration data (replace with your actual values)
    camera_matrix = np.array([[921.170702, 0.000000, 459.904354],
                              [0.000000, 919.018377, 351.238301],
                              [0.000000, 0.000000, 1.000000]])
    dist_coeffs = np.array([-0.033458, 0.105152, 0.001256, -0.006647, 0.000000])

    # Define the size of the ArUco marker in meters
    marker_length = 0.14  # aruco marker size

    # Load baseline CSV data
    baseline_csv_file = 'output.csv'  # Replace with your baseline CSV filename
    baseline_data = load_baseline_csv(baseline_csv_file)

    # Process live video and generate movement commands
    process_live_video(baseline_data)